<a href="https://colab.research.google.com/github/sattuharsha/final-year-project/blob/main/Unit_3/VLM_Openrouter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_community -q

In [ ]:
!pip install langchain-experimental -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.1/210.1 kB 12.4 MB/s eta 0:00:00


In [ ]:
!pip install langchain-openai -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.1 MB/s eta 0:00:00


In [ ]:
import os
import base64
import mimetypes
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage


os.environ["OPENAI_API_KEY"] = "api-key"
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"


MODEL = "google/gemma-3-4b-it:free"

def to_data_url(path):
    """Convert local image to base64 data URL for OpenRouter vision input."""
    mime, _ = mimetypes.guess_type(path)
    if mime is None:
        mime = "image/png"
    with open(path, "rb") as f:
        b64 = base64.b64encode(f.read()).decode("utf-8")
    return f"data:{mime};base64,{b64}"

def image_part(path_or_url: str):
    """Return correct multimodal content for OpenRouter (URL or local file)."""
    if path_or_url.startswith(("http://", "https://", "data:")):
        url = path_or_url
    else:
        url = to_data_url(path_or_url)
    return {"type": "image_url", "image_url": {"url": url}}


llm = ChatOpenAI(
    model=MODEL,
    temperature=0.2,
    max_tokens=128,
    openai_api_key=os.environ["OPENAI_API_KEY"],
    openai_api_base=os.environ["OPENAI_API_BASE"],
)


def caption(image_path_or_url: str):
    messages = [
        SystemMessage(content="You are an expert image captioning assistant."),
        HumanMessage(content=[
            {"type": "text", "text": "Describe this image in one sentence."},
            image_part(image_path_or_url),
        ])
    ]
    response = llm.invoke(messages)
    return response.content




# 1) Using an image URL
print("URL Caption:\n", caption(
    "https://images.unsplash.com/photo-1500530855697-b586d89ba3ee?w=1200"
))

# 2) Using a local file
# print("Local Caption:\n", caption("my_image.jpg"))